# 과정!

#### import, drive mount

In [1]:
pip install konlpy

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 데이터 전처리

In [4]:
wine = pd.read_csv('https://raw.githubusercontent.com/CAEL0/ybigtaTeamProject/master/Crawling/total.csv')
wine_original = wine.copy()
wine_tokenized = pd.read_csv('https://raw.githubusercontent.com/CAEL0/ybigtaTeamProject/master/Tokenizing/wine_tokenizing_final.csv')

In [5]:
# 크롤링 파일 정리
temp = wine.drop(['Unnamed: 0','name_en','country','city','aroma','food','alcohol','rate_expert','rate_consumer',"maker's_note", 'tasting_note', 'posting_title', 'posting_text', 'wine_title','wine_text','hashtag'], axis=1)
wine = temp

temp = wine.dropna(subset=['blog_text'])
temp.shape
wine = temp

# 토크나이징 파일 정리
wine_tokenized.head()
temp = wine_tokenized.drop(['Unnamed: 0'], axis=1)
wine_tokenized = temp

wine_tokenized = wine_tokenized.drop_duplicates(subset= 'name_ko', keep='first')
wine_tokenized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 0 to 324
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name_ko     305 non-null    object
 1   blogreview  305 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


#### 계절+분위기 와인 리스트 받아오기

In [6]:
def readWineList(file):
  f = open('/content/drive/MyDrive/모델링 자료/'+file+'.txt')
  list_for_return=list()
  while True:
      line = f.readline()
      if not line: 
          break
      raw = line.strip('\n"')
      list_for_return.append(raw)
  f.close()
  list_for_return = list_for_return[1:]
  return list_for_return

In [7]:
def wine_df (arr):
  result = pd.DataFrame()
  for winename in arr:
    result = result.append(wine_tokenized.loc[wine_tokenized['name_ko']==winename],ignore_index=True)
  result = result.append({'name_ko':'달콤 상큼', 'blogreview': '달콤 스위트함 달다 달콤함을 모스카토 스위트 달콤새콤함 달큰함 달달 단맛 달콤한 디저트 단맛과 달콤하다 트러스와 상큼 산도의 산미 트러스의 트러스 산도감 산도와 산미와 산도감과 산도 상큼함'}, ignore_index=True)
  result = result.append({'name_ko':'드라이 무겁', 'blogreview': '드라이 탄닌감 탄닌과 타닌감과 드라이함 타닌의 타닌 타닌과 타닌감 탄닌 타닌감의 떫 탄닌의 바디감과 파워 묵직한 무게 무겁다 바디감 바디감의 바디와 무게감 바디의 파워풀 바디 바디감은 묵직하다'}, ignore_index=True)
  return result

In [8]:
# 계절+분위기 와인 txt파일에서 리스트로 변환
categories = ['spring_rest','spring_party','summer_rest','summer_party','fall_rest','fall_party','winter_rest','winter_party']

spring_rest = readWineList('spring_rest')
spring_party = readWineList('spring_rest')
summer_rest = readWineList('summer_rest')
summer_party = readWineList('summer_party')

fall_rest = readWineList('fall_rest')
fall_party = readWineList('fall_party')
winter_rest = readWineList('winter_rest')
winter_party = readWineList('winter_party')

In [9]:
# '달콤가볍', '드라이무겁' 포함해 blogreview가 들어있는 데이터프레임으로 변환
df_spring_rest = wine_df(spring_rest)
df_spring_party = wine_df(spring_rest)
df_summer_rest = wine_df(summer_rest)
df_summer_party = wine_df(summer_party)
df_fall_rest = wine_df(fall_rest)
df_fall_party = wine_df(fall_party)
df_winter_rest = wine_df(winter_rest)
df_winter_party = wine_df(winter_party)

#### tf-idf

In [10]:
def get_recommendations(name_ko, df):
  tfidf = TfidfVectorizer()
  # blogreview에 대해 tf-idf 수행
  tfidf_matrix = tfidf.fit_transform(df['blogreview'])
  print(tfidf_matrix.shape)

  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
  return get_rcmd(name_ko,df,cosine_sim)

def get_rcmd (name_ko, df, cosine_sim) :

  indices = pd.Series(df.index, index=df['name_ko'])
  # 선택한 와인의 이름으로부터 해당되는 인덱스를 받아옴
  idx = indices[name_ko]
  
  # 모든 와인에 대해 해당 표본과의 유사도를 구함
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 유사도에 따라 와인들 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 가장 유사한 10개의 와인 받아옴
  sim_scores = sim_scores[1:21]

  # 가장 유사한 10개의 와인의 인덱스를 받아옴
  wine_indices = [i[0] for i in sim_scores]

  # 결과 값 리턴
  return df['name_ko'].iloc[wine_indices]

In [11]:
def returnWineInfo (arr):
  result = pd.DataFrame()
  for winename in arr:
    result = result.append(wine.loc[wine['name_ko']==winename],ignore_index=True)
  return result

In [12]:
def returnBlogReview (arr):
  result = pd.DataFrame()
  for winename in arr:
    result = result.append(wine_tokenized.loc[wine_tokenized['name_ko']==winename],ignore_index=True)
  return result

# 최종 결과!
총 16가지 결과를 출력해놓았습니다:)

* 질문 별 사용된 워딩

  - 계절 - spring, summer, fall, winter
  - 분위기 - rest, party
  - 맛 - 달콤&상큼, 드라이&무겁



* 맨 첫 줄 주석에 유형이 나와있습니다.

  - 유형 별로 첫번째 셀은 크롤링 데이터에서 추출한 내용으로, 가격 및 기타 와인 정보가 들어 있습니다.
 - 두번째 셀은 토크나이징 데이터에서 추출한 내용으로, 워드클라우드 제작을 위한 blogreview가 들어 있습니다.

## 봄

In [13]:
# 봄, 힐링, 스위트
spring_rest_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_spring_rest).unique())
spring_rest_sweet

(282, 19234)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"포르투 발도우로, 화이트 포트",30000.0,NaN,5.0,3.0,5.0,1.0,"포르토, 발로로 화이트 포트,,\n\n주정강화 와인을 좋아하진 않지만,\n다양성 차..."
2,트라이마리,42000.0,스파클링,3.0,2.0,2.0,1.0,*DUCA DI CASTEMONTE\nTRAIMARI VINO FRIZZANTE\n...
3,"도미니오 데 라 베가, 디 엔드",45000.0,NaN,5.0,3.0,4.0,1.0,스페인 와인\n[스페인와인] 도미니오 데 라 베가 디 엔드 / DOMINIO DE ...
4,"에라스모, 로제 드 무르베드르",56000.0,NaN,1.0,4.0,2.0,1.0,BEST와인추천\n와인21닷컴이 선정한 6월 이달의 와인\n와인21 ・ 2021. ...
5,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",19800.0,NaN,1.0,4.0,2.0,1.0,2만원 이하\n얄리 프리미엄 릴리즈 언오크드 샤르도네 2018 Yali Premiu...
6,"캐년 오크, 샤르도네",25000.0,NaN,1.0,3.0,3.0,1.0,US\n[와인/미국] Canyon Oaks Chardonnay 2018\nJerry...
7,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...


In [14]:
returnBlogReview(get_recommendations('달콤 상큼',df_spring_rest).unique())

(282, 19234)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"포르투 발도우로, 화이트 포트",과일 포트 생산자 오크 익은 숙성 기분 좋은 산미 견과류 화이트 등급 옅은 볏짚 색...
2,트라이마리,시칠리아 아로마 생산 트러스 토착 품종 스파클링 미네랄 신선한 카를로 펠레 화이트 ...
3,"도미니오 데 라 베가, 디 엔드",발렌시아 포도 미와 신선한 산미 지역 달콤한 트러스 파인애플 입 가득 흰 풍부한 좋...
4,"에라스모, 로제 드 무르베드르",향 배 딸기 장미 트러스 오렌지 느낄 꽃 빛 과일 강하게 일어나며 그레이 프 푸르츠...
5,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",향 포도 허브 레몬 자몽 트러스 오렌지 미네랄 산뜻한 산도 접촉 생산 과일 바디 신...
6,"캐년 오크, 샤르도네",오크 사과 복숭아 바닐라 멜론 아로마 긴 여운 과실 향 품질 금빛 감도는 볏짚 색 ...
7,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...


In [15]:
# 봄, 힐링, 드라이
spring_rest_dry = returnWineInfo(get_recommendations('드라이 무겁',df_spring_rest).unique())
spring_rest_dry

(282, 19234)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"에쉔호프 홀져, 더 오렌지",100000.0,NaN,1.0,5.0,2.0,1.0,Wine 이야기\n에쉔호프 홀져 더 오렌지 2017 (Eschenhof Holzer...
1,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
2,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
3,"제라르 베르트랑, 꼬뜨 로즈 로제",33000.0,NaN,1.0,3.0,3.0,1.0,ROSE WINE\n[Gerard Bertrand] Cote des Roses Ro...


In [16]:
returnBlogReview(get_recommendations('드라이 무겁',df_spring_rest).unique())

(282, 19234)


,name_ko,blogreview
0,"에쉔호프 홀져, 더 오렌지",오렌지 아몬드 살구 홍차 숙성 오스트리아 내추럴 투명한 차 향 절 느낌 어우러져 스...
1,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
2,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
3,"제라르 베르트랑, 꼬뜨 로즈 로제",장미 은은한 향 체리 꽃 과실 싱그러운 하여 병 살구 살구색 콤 풍부한 붉은 향기 ...


In [17]:
# 봄, 파티, 스위트
spring_party_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_spring_party).unique())
spring_party_sweet

(282, 19234)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"포르투 발도우로, 화이트 포트",30000.0,NaN,5.0,3.0,5.0,1.0,"포르토, 발로로 화이트 포트,,\n\n주정강화 와인을 좋아하진 않지만,\n다양성 차..."
2,트라이마리,42000.0,스파클링,3.0,2.0,2.0,1.0,*DUCA DI CASTEMONTE\nTRAIMARI VINO FRIZZANTE\n...
3,"도미니오 데 라 베가, 디 엔드",45000.0,NaN,5.0,3.0,4.0,1.0,스페인 와인\n[스페인와인] 도미니오 데 라 베가 디 엔드 / DOMINIO DE ...
4,"에라스모, 로제 드 무르베드르",56000.0,NaN,1.0,4.0,2.0,1.0,BEST와인추천\n와인21닷컴이 선정한 6월 이달의 와인\n와인21 ・ 2021. ...
5,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",19800.0,NaN,1.0,4.0,2.0,1.0,2만원 이하\n얄리 프리미엄 릴리즈 언오크드 샤르도네 2018 Yali Premiu...
6,"캐년 오크, 샤르도네",25000.0,NaN,1.0,3.0,3.0,1.0,US\n[와인/미국] Canyon Oaks Chardonnay 2018\nJerry...
7,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...


In [18]:
returnBlogReview(get_recommendations('달콤 상큼',df_spring_party).unique())

(282, 19234)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"포르투 발도우로, 화이트 포트",과일 포트 생산자 오크 익은 숙성 기분 좋은 산미 견과류 화이트 등급 옅은 볏짚 색...
2,트라이마리,시칠리아 아로마 생산 트러스 토착 품종 스파클링 미네랄 신선한 카를로 펠레 화이트 ...
3,"도미니오 데 라 베가, 디 엔드",발렌시아 포도 미와 신선한 산미 지역 달콤한 트러스 파인애플 입 가득 흰 풍부한 좋...
4,"에라스모, 로제 드 무르베드르",향 배 딸기 장미 트러스 오렌지 느낄 꽃 빛 과일 강하게 일어나며 그레이 프 푸르츠...
5,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",향 포도 허브 레몬 자몽 트러스 오렌지 미네랄 산뜻한 산도 접촉 생산 과일 바디 신...
6,"캐년 오크, 샤르도네",오크 사과 복숭아 바닐라 멜론 아로마 긴 여운 과실 향 품질 금빛 감도는 볏짚 색 ...
7,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...


In [19]:
# 봄, 파티, 드라이
spring_party_dry = returnWineInfo(get_recommendations('드라이 무겁',df_spring_party).unique())
spring_party_dry

(282, 19234)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"에쉔호프 홀져, 더 오렌지",100000.0,NaN,1.0,5.0,2.0,1.0,Wine 이야기\n에쉔호프 홀져 더 오렌지 2017 (Eschenhof Holzer...
1,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
2,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
3,"제라르 베르트랑, 꼬뜨 로즈 로제",33000.0,NaN,1.0,3.0,3.0,1.0,ROSE WINE\n[Gerard Bertrand] Cote des Roses Ro...


In [20]:
returnBlogReview(get_recommendations('드라이 무겁',df_spring_party).unique())

(282, 19234)


,name_ko,blogreview
0,"에쉔호프 홀져, 더 오렌지",오렌지 아몬드 살구 홍차 숙성 오스트리아 내추럴 투명한 차 향 절 느낌 어우러져 스...
1,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
2,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
3,"제라르 베르트랑, 꼬뜨 로즈 로제",장미 은은한 향 체리 꽃 과실 싱그러운 하여 병 살구 살구색 콤 풍부한 붉은 향기 ...


## 여름

In [21]:
# 여름, 힐링, 스위트
summer_rest_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_summer_rest).unique())
summer_rest_sweet

(382, 19787)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"아멜리아 파크, 리저브 샤도네이",175000.0,NaN,1.0,4.0,4.0,1.0,BEST와인추천\n와인21닷컴이 선정한 8월 이달의 와인\n와인21 ・ 2021. ...
2,"또스띠, 모스카토 다스티 하트",35000.0,NaN,4.0,2.0,2.0,1.0,달콤 스위트와인\n또스띠 똘라스코 모스까또 다스티 Tosti Torlasco Mos...
3,트라이마리,42000.0,스파클링,3.0,2.0,2.0,1.0,*DUCA DI CASTEMONTE\nTRAIMARI VINO FRIZZANTE\n...
4,"에라스모, 로제 드 무르베드르",56000.0,NaN,1.0,4.0,2.0,1.0,BEST와인추천\n와인21닷컴이 선정한 6월 이달의 와인\n와인21 ・ 2021. ...


In [22]:
returnBlogReview(get_recommendations('달콤 상큼',df_summer_rest).unique())

(382, 19787)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"아멜리아 파크, 리저브 샤도네이",꽃 자몽 감귤 미네랄 프리미엄 버터 아로마 도 산도 장기 숙성 가능한 서호주 화이트...
2,"또스띠, 모스카토 다스티 하트",꽃 모스 복숭아 살구 향 알코올 카토 레몬 오렌지 보리수 특유 과일 도수 밸런타인데...
3,트라이마리,시칠리아 아로마 생산 트러스 토착 품종 스파클링 미네랄 신선한 카를로 펠레 화이트 ...
4,"에라스모, 로제 드 무르베드르",향 배 딸기 장미 트러스 오렌지 느낄 꽃 빛 과일 강하게 일어나며 그레이 프 푸르츠...


In [23]:
# 여름, 힐링, 드라이
summer_rest_dry = returnWineInfo(get_recommendations('드라이 무겁',df_summer_rest).unique())
summer_rest_dry

(382, 19787)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
1,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
2,"메이오미, 피노 누아",90000.0,NaN,1.0,3.0,3.0,3.0,"Tasting Diary\nMeiomi, Pinot Noir 2018 (메이오미, ..."
3,"마르케시 디 바롤로, 랑게 브릭 아멜",78000.0,NaN,1.0,4.0,3.0,1.0,"Italy\nMarchesi di Barolo, Sarmassa Barolo DOC..."
4,"제라르 베르트랑, 꼬뜨 로즈 로제",33000.0,NaN,1.0,3.0,3.0,1.0,ROSE WINE\n[Gerard Bertrand] Cote des Roses Ro...
5,알렉산드린 비오니에,50000.0,NaN,1.0,2.0,2.0,1.0,ᴡɪɴᴇ\n[와인을 마시고 기록해보자] 3. 메종 레 알렉산드린 비오니에 2018 ...
6,"엠 샤푸티에, 데샹트 생 조셉 '아티스트 레이블'",120000.0,NaN,1.0,3.0,4.0,4.0,와인 후기\n엠 샤푸티에 데샹트 생 조셉 2015 M. Chapoutier Desc...


In [24]:
returnBlogReview(get_recommendations('드라이 무겁',df_summer_rest).unique())

(382, 19787)


,name_ko,blogreview
0,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
1,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
2,"메이오미, 피노 누아",카운티 누 빈야드 향 자두 오크 캘리포니아 포도 양조 진행 는 와 체리 딸기 홍차 ...
3,"마르케시 디 바롤로, 랑게 브릭 아멜",품종 꿀 포도 가지 각 아몬드 복숭아 살구 꽃 아카시아 지역 과 뜻 합쳐진 이름 향...
4,"제라르 베르트랑, 꼬뜨 로즈 로제",장미 은은한 향 체리 꽃 과실 싱그러운 하여 병 살구 살구색 콤 풍부한 붉은 향기 ...
5,알렉산드린 비오니에,북론 복숭아 좋은 세 사람 살구 흰 진한 아로마 생기 산뜻한 6 5월 알렉산 지역 ...
6,"엠 샤푸티에, 데샹트 생 조셉 '아티스트 레이블'",향 감초 꽃 향신료 엠 아티스트 배 향기 두드러지며 시간 지나면서 점점 올라온다 지...


In [25]:
# 여름, 파티, 스위트
summer_party_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_summer_party).unique())
summer_party_sweet

(332, 21527)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"아멜리아 파크, 리저브 샤도네이",175000.0,NaN,1.0,4.0,4.0,1.0,BEST와인추천\n와인21닷컴이 선정한 8월 이달의 와인\n와인21 ・ 2021. ...
2,"또스띠, 모스카토 다스티 하트",35000.0,NaN,4.0,2.0,2.0,1.0,달콤 스위트와인\n또스띠 똘라스코 모스까또 다스티 Tosti Torlasco Mos...
3,트라이마리,42000.0,스파클링,3.0,2.0,2.0,1.0,*DUCA DI CASTEMONTE\nTRAIMARI VINO FRIZZANTE\n...
4,"에라스모, 로제 드 무르베드르",56000.0,NaN,1.0,4.0,2.0,1.0,BEST와인추천\n와인21닷컴이 선정한 6월 이달의 와인\n와인21 ・ 2021. ...
5,"프론테라, 프리미엄 스파클링 스위트 로제",14900.0,스파클링,1.0,5.0,2.0,1.0,BEST와인추천\n와인21닷컴이 선정한 7월 이달의 와인\n와인21 ・ 2021. ...
6,"브라이다, 지 모스카토 다스티",40000.0,NaN,4.0,2.0,2.0,1.0,놀이방\n본격 와인리뷰- Braida G moscato d'asti(브라이다 지 모...
7,"마스카 델 타코, 로시 피노 네로 로사토",38000.0,NaN,1.0,3.0,2.0,1.0,이탈리아와인\nMASCA DEL TACCO [마스카 델 타코] 제이픽와인\n산타 ・...
8,"로클랜드, 초콜릿 박스 스파클링 쉬라즈",38000.0,스파클링,2.0,2.0,4.0,3.0,AUSTRALIA/NEWZE\n[Rocland] Chocolate Box _Cabe...
9,"도미니오 데 라 베가, 디 엔드",45000.0,NaN,5.0,3.0,4.0,1.0,스페인 와인\n[스페인와인] 도미니오 데 라 베가 디 엔드 / DOMINIO DE ...


In [26]:
returnBlogReview(get_recommendations('달콤 상큼',df_summer_party).unique())

(332, 21527)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"아멜리아 파크, 리저브 샤도네이",꽃 자몽 감귤 미네랄 프리미엄 버터 아로마 도 산도 장기 숙성 가능한 서호주 화이트...
2,"또스띠, 모스카토 다스티 하트",꽃 모스 복숭아 살구 향 알코올 카토 레몬 오렌지 보리수 특유 과일 도수 밸런타인데...
3,트라이마리,시칠리아 아로마 생산 트러스 토착 품종 스파클링 미네랄 신선한 카를로 펠레 화이트 ...
4,"에라스모, 로제 드 무르베드르",향 배 딸기 장미 트러스 오렌지 느낄 꽃 빛 과일 강하게 일어나며 그레이 프 푸르츠...
5,"프론테라, 프리미엄 스파클링 스위트 로제",스파클링 디저트 식전주 해산물 샐러드 옅은 어색 부드럽고 섬세한 포 신선한 붉은 과...
6,"브라이다, 지 모스카토 다스티",포도 향 모스 카토 는 디저트 초콜렛 과일 탐스러운 노란 색상 사랑스러운 버블 지닌...
7,"마스카 델 타코, 로시 피노 네로 로사토",향 딸기 미네랄 밟은 옅은 분홍색 야생 젤리 수박 달콤한 풍부하고 그 향신료 광물 ...
8,"로클랜드, 초콜릿 박스 스파클링 쉬라즈",초콜릿 자두 과실 향 타닌 레이블 스파클링 검은 류 부드러운 바닐라 입힌 딸기 느낌...
9,"도미니오 데 라 베가, 디 엔드",발렌시아 포도 미와 신선한 산미 지역 달콤한 트러스 파인애플 입 가득 흰 풍부한 좋...


In [27]:
# 여름, 파티, 드라이
summer_party_dry = returnWineInfo(get_recommendations('드라이 무겁',df_summer_party).unique())
summer_party_dry

(332, 21527)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
1,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
2,"머드 하우스, 피노 누아",70000.0,NaN,1.0,4.0,3.0,3.0,"레드와인\n[레드] 머드 하우스, 피노 누아 2018 Mud House, Centr..."
3,"메이오미, 피노 누아",90000.0,NaN,1.0,3.0,3.0,3.0,"Tasting Diary\nMeiomi, Pinot Noir 2018 (메이오미, ..."
4,"피에르 가이야르, 생 조셉 루즈",90000.0,NaN,1.0,3.0,5.0,4.0,Vin\npierre gaillard saint-joseph blanc\n한상 ・ ...
5,"로르나노, 키안티 클라시코 그랑 셀레지오네",100000.0,NaN,1.0,4.0,4.0,4.0,와인이야기\n#Castello di Meleto Chianti Classico Gr...
6,"루이스 엠 마티니, 나파 밸리",NaN,NaN,1.0,4.0,4.0,4.0,"와인 리뷰\n[와인] 루티 마티니, 나파밸리 까베르네 소비뇽 2016 (Louis ..."
7,"마르케시 디 바롤로, 랑게 브릭 아멜",78000.0,NaN,1.0,4.0,3.0,1.0,"Italy\nMarchesi di Barolo, Sarmassa Barolo DOC..."
8,"마르케시 디 바롤로, 네비올로 달바 미쉐트",NaN,NaN,1.0,4.0,3.0,4.0,Red\n와인앤모어 탐방기-혼술용 이태리 레드와인(마르께시 디 바롤로 네비올로 달바...


In [28]:
returnBlogReview(get_recommendations('드라이 무겁',df_summer_party).unique())

(332, 21527)


,name_ko,blogreview
0,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
1,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
2,"머드 하우스, 피노 누아",오크 향 체리 레드 숙성 아름다운 루비 빛 컬러 라즈베리 얻어진 스파이 시한 향신료...
3,"메이오미, 피노 누아",카운티 누 빈야드 향 자두 오크 캘리포니아 포도 양조 진행 는 와 체리 딸기 홍차 ...
4,"피에르 가이야르, 생 조셉 루즈",라스베리 토스트 과일 느낌 블랙 베리 향 스모키 맑고 투명하게 느껴지면서도 익은 맛...
5,"로르나노, 키안티 클라시코 그랑 셀레지오네",자두 체리 블랙베리 정향 감초 검은 향 깊고 진한 루비 색 류 과일 각종 스파이 시...
6,"루이스 엠 마티니, 나파 밸리",체리 정향 감초 오크 카라멜 삼나무 블랙 맛 힌트 진한 버건디 색상 베리 아로마 뉘...
7,"마르케시 디 바롤로, 랑게 브릭 아멜",품종 꿀 포도 가지 각 아몬드 복숭아 살구 꽃 아카시아 지역 과 뜻 합쳐진 이름 향...
8,"마르케시 디 바롤로, 네비올로 달바 미쉐트",향 포도 오크 풍부한 바닐라 미네랄 껍질 탱크 숙성 피에몬테 지역 토양 자란 비올 ...


## 가을

In [29]:
# 가을, 힐링, 스위트
fall_rest_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_fall_rest).unique())
fall_rest_sweet

(112, 15686)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",19800.0,NaN,1.0,4.0,2.0,1.0,2만원 이하\n얄리 프리미엄 릴리즈 언오크드 샤르도네 2018 Yali Premiu...
2,"그랜트 버지, 쏜 리슬링",70000.0,NaN,1.0,3.0,3.0,1.0,Wine Whiskey\n호주 Australia 화이트 와인 White Wine 그...
3,"보시오, 트로피칼 모스카토 패션 후르츠",24000.0,NaN,4.0,2.0,2.0,1.0,일상\n이마트24 편의점 달달한 스파클링 와인추천 [보시오 트로피칼 모스카토 패션후...
4,"실레니, 셀라 셀렉션 스파클링 소비뇽 블랑",60000.0,스파클링,1.0,3.0,3.0,1.0,"스파클링와인/샴페인\n[스파클링] 실레니, 셀라 셀렉션 스파클링 소비뇽 블랑 NV ..."
5,"고세, 그랑 로제 브뤼",220000.0,스파클링,1.0,4.0,3.0,1.0,"프랑스 스파클링와인 샴페인\n고세,그랑 로제 브뤼\nGosset, Grand Ros..."
6,"도멘 라파주, 꼬떼 플로랄",21000.0,NaN,1.0,3.0,3.0,1.0,"[프랑스] Domaine Lafage Cote Floral, IGP Cotes Ca..."
7,"캐년 오크, 샤르도네",25000.0,NaN,1.0,3.0,3.0,1.0,US\n[와인/미국] Canyon Oaks Chardonnay 2018\nJerry...
8,프로메사 모스카토,17000.0,NaN,3.0,3.0,1.0,1.0,"알콜과 음악\n[Promesa, moscato] 프로메사 모스카토 750ml\nPE..."
9,"비솔, 제이오 발도비아데네 프로세코 수페리오레 엑스트라 드라이",70000.0,스파클링,3.0,3.0,2.0,1.0,"ITALY\n[Bisol] Prosecco Superiore DOCG Jeio, C..."


In [43]:
returnBlogReview(get_recommendations('달콤 상큼',df_fall_rest).unique())

(112, 15686)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",향 포도 허브 레몬 자몽 트러스 오렌지 미네랄 산뜻한 산도 접촉 생산 과일 바디 신...
2,"그랜트 버지, 쏜 리슬링",리 슬링 생산 포도 지역 미네랄 에덴 호주 바로 사 레몬 꽃 좋고 5 것 그랜트 버...
3,"보시오, 트로피칼 모스카토 패션 후르츠",모스 카토 발효 꽃 과실 도 청포도 향 긋한 미를 과정 과일 24시간 만든 인기 사...
4,"실레니, 셀라 셀렉션 스파클링 소비뇽 블랑",스파클링 소비 뇽 레몬 블랑 마시기 라임 피망 향 느낌 매력 좋은 데 뉴질랜드 살짝...
5,"고세, 그랑 로제 브뤼",향 로제 샴페인 고세 과일 잔 체리 딸기 시나몬 육두구 버터 반짝이는 연한 방울 커...
6,"도멘 라파주, 꼬떼 플로랄",꽃 향 미네랄 신선한 과실 프랑스 복숭아 라임 이름 아로마 라파 남 맑고 밝은 연두...
7,"캐년 오크, 샤르도네",오크 사과 복숭아 바닐라 멜론 아로마 긴 여운 과실 향 품질 금빛 감도는 볏짚 색 ...
8,프로메사 모스카토,서양배 사과 트러스 망고 구아바 아로마 적당한 도 포 상큼 함 기분 좋은 청량감 선...
9,"비솔, 제이오 발도비아데네 프로세코 수페리오레 엑스트라 드라이",프로세코 꽃 발도 생산 비솔 향 과일 비아데네 페리 오레 생동감 넘치는 버블 적절한...


In [30]:
# 가을, 힐링, 드라이
fall_rest_dry = returnWineInfo(get_recommendations('드라이 무겁',df_fall_rest).unique())
fall_rest_dry

(112, 15686)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
1,"레 파셀레르 드 쏘, 쥬브리 샹베르탱",269000.0,NaN,1.0,4.0,4.0,3.0,BEST와인추천\n와인21닷컴이 선정한 10월 이달의 와인\n와인21 ・ 2020....
2,"레이몬드, 리저브 셀렉션 메를로",190000.0,NaN,1.0,2.0,4.0,3.0,wine\n40주년 한정판 와인 : 레이몬드 리저브 셀렉션 까베르네 소비뇽 2018...
3,"파밀리아 마로네, 랑게 샤도네이 '메문디스'",60000.0,NaN,1.0,4.0,4.0,1.0,"Italy\nFamiglia Marrone, Memundis Chardonnay, ..."
4,"탈레, 트레비아노 다브루쪼",100000.0,NaN,1.0,3.0,3.0,3.0,Italy\nCasal Thaulero 'Thale' Trebbiano d'Abru...
5,"샤또 푸에슈오, 프레스티지 루즈",108000.0,NaN,1.0,3.0,4.0,4.0,프랑스 와인\n[제이와인샵#] - 샤또 푸에슈오 프레스티지 루즈 2013 ( Cha...
6,"비솔, 제이오 발도비아데네 프로세코 수페리오레 엑스트라 드라이",70000.0,스파클링,3.0,3.0,2.0,1.0,"ITALY\n[Bisol] Prosecco Superiore DOCG Jeio, C..."
7,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",19800.0,NaN,1.0,4.0,2.0,1.0,2만원 이하\n얄리 프리미엄 릴리즈 언오크드 샤르도네 2018 Yali Premiu...


In [31]:
returnBlogReview(get_recommendations('드라이 무겁',df_fall_rest).unique())

(112, 15686)


,name_ko,blogreview
0,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
1,"레 파셀레르 드 쏘, 쥬브리 샹베르탱",드 블랙베리 오크 강렬하면서도 우아한 복합 섬세한 질감 오트 쿠튀르 하우스 레 파셀...
2,"레이몬드, 리저브 셀렉션 메를로",자두 블루베리 블랙 커런트 오크 18 개월 정도 프랑스 산 통 숙성 리저 브 메를 ...
3,"파밀리아 마로네, 랑게 샤도네이 '메문디스'",오크 향 통 바닐라 생산 과일 화이트 마로 발효 사과 꽃 아카시아 숙성 느껴지는 고...
4,"탈레, 트레비아노 다브루쪼",꽃 배 아카시아 향기 대상 발사믹 힌트 11월 대한민국 주류 화이트 레 지역 전 깊...
5,"샤또 푸에슈오, 프레스티지 루즈",자두 석류 딸기 검 붉은 색 산딸기 키르 쉬 버찌 나무 초목 향등 아로마 조화롭게 ...
6,"비솔, 제이오 발도비아데네 프로세코 수페리오레 엑스트라 드라이",프로세코 꽃 발도 생산 비솔 향 과일 비아데네 페리 오레 생동감 넘치는 버블 적절한...
7,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",향 포도 허브 레몬 자몽 트러스 오렌지 미네랄 산뜻한 산도 접촉 생산 과일 바디 신...


In [32]:
# 가을, 파티, 스위트
fall_party_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_fall_party).unique())
fall_party_sweet

(132, 13924)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",19800.0,NaN,1.0,4.0,2.0,1.0,2만원 이하\n얄리 프리미엄 릴리즈 언오크드 샤르도네 2018 Yali Premiu...
2,"그랜트 버지, 쏜 리슬링",70000.0,NaN,1.0,3.0,3.0,1.0,Wine Whiskey\n호주 Australia 화이트 와인 White Wine 그...
3,"실레니, 익셉셔널 셀렉션 소비뇽 블랑",78000.0,NaN,1.0,4.0,2.0,1.0,2020년\nSileni Exceptional Selection Sauvignon ...
4,"보시오, 트로피칼 모스카토 패션 후르츠",24000.0,NaN,4.0,2.0,2.0,1.0,일상\n이마트24 편의점 달달한 스파클링 와인추천 [보시오 트로피칼 모스카토 패션후...
5,"고세, 그랑 로제 브뤼",220000.0,스파클링,1.0,4.0,3.0,1.0,"프랑스 스파클링와인 샴페인\n고세,그랑 로제 브뤼\nGosset, Grand Ros..."
6,"실레니, 셀라 셀렉션 스파클링 소비뇽 블랑",60000.0,스파클링,1.0,3.0,3.0,1.0,"스파클링와인/샴페인\n[스파클링] 실레니, 셀라 셀렉션 스파클링 소비뇽 블랑 NV ..."
7,"도멘 라파주, 꼬떼 플로랄",21000.0,NaN,1.0,3.0,3.0,1.0,"[프랑스] Domaine Lafage Cote Floral, IGP Cotes Ca..."
8,"캐년 오크, 샤르도네",25000.0,NaN,1.0,3.0,3.0,1.0,US\n[와인/미국] Canyon Oaks Chardonnay 2018\nJerry...
9,프로메사 모스카토,17000.0,NaN,3.0,3.0,1.0,1.0,"알콜과 음악\n[Promesa, moscato] 프로메사 모스카토 750ml\nPE..."


In [33]:
returnBlogReview(get_recommendations('달콤 상큼',df_fall_party).unique())

(132, 13924)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",향 포도 허브 레몬 자몽 트러스 오렌지 미네랄 산뜻한 산도 접촉 생산 과일 바디 신...
2,"그랜트 버지, 쏜 리슬링",리 슬링 생산 포도 지역 미네랄 에덴 호주 바로 사 레몬 꽃 좋고 5 것 그랜트 버...
3,"실레니, 익셉셔널 셀렉션 소비뇽 블랑",소비 뇽 블랑 복숭아 라임 자몽 리치 말보로 하여 는 남쪽 지역 파이 밸리 자라는 ...
4,"보시오, 트로피칼 모스카토 패션 후르츠",모스 카토 발효 꽃 과실 도 청포도 향 긋한 미를 과정 과일 24시간 만든 인기 사...
5,"고세, 그랑 로제 브뤼",향 로제 샴페인 고세 과일 잔 체리 딸기 시나몬 육두구 버터 반짝이는 연한 방울 커...
6,"실레니, 셀라 셀렉션 스파클링 소비뇽 블랑",스파클링 소비 뇽 레몬 블랑 마시기 라임 피망 향 느낌 매력 좋은 데 뉴질랜드 살짝...
7,"도멘 라파주, 꼬떼 플로랄",꽃 향 미네랄 신선한 과실 프랑스 복숭아 라임 이름 아로마 라파 남 맑고 밝은 연두...
8,"캐년 오크, 샤르도네",오크 사과 복숭아 바닐라 멜론 아로마 긴 여운 과실 향 품질 금빛 감도는 볏짚 색 ...
9,프로메사 모스카토,서양배 사과 트러스 망고 구아바 아로마 적당한 도 포 상큼 함 기분 좋은 청량감 선...


In [34]:
# 가을, 파티, 드라이
fall_party_dry = returnWineInfo(get_recommendations('드라이 무겁',df_fall_party).unique())
fall_party_dry

(132, 13924)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
1,"레 파셀레르 드 쏘, 쥬브리 샹베르탱",269000.0,NaN,1.0,4.0,4.0,3.0,BEST와인추천\n와인21닷컴이 선정한 10월 이달의 와인\n와인21 ・ 2020....
2,"레이몬드, 리저브 셀렉션 메를로",190000.0,NaN,1.0,2.0,4.0,3.0,wine\n40주년 한정판 와인 : 레이몬드 리저브 셀렉션 까베르네 소비뇽 2018...
3,"파밀리아 마로네, 랑게 샤도네이 '메문디스'",60000.0,NaN,1.0,4.0,4.0,1.0,"Italy\nFamiglia Marrone, Memundis Chardonnay, ..."
4,"탈레, 트레비아노 다브루쪼",100000.0,NaN,1.0,3.0,3.0,3.0,Italy\nCasal Thaulero 'Thale' Trebbiano d'Abru...


In [35]:
returnBlogReview(get_recommendations('드라이 무겁',df_fall_party).unique())

(132, 13924)


,name_ko,blogreview
0,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
1,"레 파셀레르 드 쏘, 쥬브리 샹베르탱",드 블랙베리 오크 강렬하면서도 우아한 복합 섬세한 질감 오트 쿠튀르 하우스 레 파셀...
2,"레이몬드, 리저브 셀렉션 메를로",자두 블루베리 블랙 커런트 오크 18 개월 정도 프랑스 산 통 숙성 리저 브 메를 ...
3,"파밀리아 마로네, 랑게 샤도네이 '메문디스'",오크 향 통 바닐라 생산 과일 화이트 마로 발효 사과 꽃 아카시아 숙성 느껴지는 고...
4,"탈레, 트레비아노 다브루쪼",꽃 배 아카시아 향기 대상 발사믹 힌트 11월 대한민국 주류 화이트 레 지역 전 깊...


## 겨울

In [36]:
# 겨울, 힐링, 스위트
winter_rest_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_winter_rest).unique())
winter_rest_sweet

(262, 20406)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"포르투 발도우로, 화이트 포트",30000.0,NaN,5.0,3.0,5.0,1.0,"포르토, 발로로 화이트 포트,,\n\n주정강화 와인을 좋아하진 않지만,\n다양성 차..."
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",65000.0,NaN,1.0,3.0,4.0,3.0,프랑스와인\n샤또 롱보 꼬뜨 뒤 루씨옹 뀌베 삐에르 드 라 파브레귀2016(Chat...
3,"브라이다, 지 모스카토 다스티",40000.0,NaN,4.0,2.0,2.0,1.0,놀이방\n본격 와인리뷰- Braida G moscato d'asti(브라이다 지 모...
4,"도미니오 데 라 베가, 디 엔드",45000.0,NaN,5.0,3.0,4.0,1.0,스페인 와인\n[스페인와인] 도미니오 데 라 베가 디 엔드 / DOMINIO DE ...
5,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",19800.0,NaN,1.0,4.0,2.0,1.0,2만원 이하\n얄리 프리미엄 릴리즈 언오크드 샤르도네 2018 Yali Premiu...
6,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
7,"떼땅져, 녹턴 시티 라이트 '루미너스'",319000.0,스파클링,2.0,3.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 12월 이달의 와인\n와인21 ・ 2020....


In [44]:
returnBlogReview(get_recommendations('달콤 상큼',df_winter_rest).unique())

(262, 20406)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"포르투 발도우로, 화이트 포트",과일 포트 생산자 오크 익은 숙성 기분 좋은 산미 견과류 화이트 등급 옅은 볏짚 색...
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",생산 호두 감초 타닌 오크 젤리 설탕 린 향 묵직한 무게 감 밸런스 5년 드 부드러...
3,"브라이다, 지 모스카토 다스티",포도 향 모스 카토 는 디저트 초콜렛 과일 탐스러운 노란 색상 사랑스러운 버블 지닌...
4,"도미니오 데 라 베가, 디 엔드",발렌시아 포도 미와 신선한 산미 지역 달콤한 트러스 파인애플 입 가득 흰 풍부한 좋...
5,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",향 포도 허브 레몬 자몽 트러스 오렌지 미네랄 산뜻한 산도 접촉 생산 과일 바디 신...
6,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
7,"떼땅져, 녹턴 시티 라이트 '루미너스'",건포도 복숭아 살구 꽃 트러스 향 과일 느껴지며 부드럽고 미감 샴페인 포도 스파클링...


In [37]:
# 겨울, 힐링, 드라이
winter_rest_dry = returnWineInfo(get_recommendations('드라이 무겁',df_winter_rest).unique())
winter_rest_dry

(262, 20406)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"폴 자불레 애네, 크로즈 에르미타주 '도멘 드 탈라베'",135000.0,NaN,1.0,3.0,4.0,4.0,"2권\nPaul Jaboulet Aine, Crozes Hermitage Domai..."
1,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",65000.0,NaN,1.0,3.0,4.0,3.0,프랑스와인\n샤또 롱보 꼬뜨 뒤 루씨옹 뀌베 삐에르 드 라 파브레귀2016(Chat...
3,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
4,"아그리콜라 푸니카, 바루아",140000.0,NaN,1.0,3.0,4.0,4.0,"Agricola Punica, Barrua, Isola dei Nuraghi, Sa..."


In [38]:
returnBlogReview(get_recommendations('드라이 무겁',df_winter_rest).unique())

(262, 20406)


,name_ko,blogreview
0,"폴 자불레 애네, 크로즈 에르미타주 '도멘 드 탈라베'",향 오크 가죽 는 론 시옹 가장 맑고 밝은 톤 루비 색 강렬하고 복합 붉은 열매 과...
1,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",생산 호두 감초 타닌 오크 젤리 설탕 린 향 묵직한 무게 감 밸런스 5년 드 부드러...
3,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
4,"아그리콜라 푸니카, 바루아",자두 체리 블랙베리 감초 허브 세이지 로즈마리 향 완벽한 익은 야생 블랙 올리브 미...


In [39]:
# 겨울, 파티, 스위트
winer_party_sweet = returnWineInfo(get_recommendations('달콤 상큼',df_winter_party).unique())
winer_party_sweet

(202, 17954)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,크리스마스 아스티,30000.0,스파클링,4.0,2.0,2.0,1.0,와인/위스키\n모스카토 다스티 추천 - 크리스마스 아스티 (Christmas Ast...
1,"에라스모, 로제 드 무르베드르",56000.0,NaN,1.0,4.0,2.0,1.0,BEST와인추천\n와인21닷컴이 선정한 6월 이달의 와인\n와인21 ・ 2021. ...
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",65000.0,NaN,1.0,3.0,4.0,3.0,프랑스와인\n샤또 롱보 꼬뜨 뒤 루씨옹 뀌베 삐에르 드 라 파브레귀2016(Chat...
3,"도미니오 데 라 베가, 디 엔드",45000.0,NaN,5.0,3.0,4.0,1.0,스페인 와인\n[스페인와인] 도미니오 데 라 베가 디 엔드 / DOMINIO DE ...
4,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
5,"떼땅져, 녹턴 시티 라이트 '루미너스'",319000.0,스파클링,2.0,3.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 12월 이달의 와인\n와인21 ・ 2020....
6,"떼땅져, 녹턴 시티 라이트",257000.0,스파클링,2.0,4.0,3.0,1.0,"[W]ine\n22. 샴페인: 떼땅져 녹턴 시티 라이트 (Taittinger, No..."
7,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",19800.0,NaN,1.0,4.0,2.0,1.0,2만원 이하\n얄리 프리미엄 릴리즈 언오크드 샤르도네 2018 Yali Premiu...
8,"그랜트 버지, 쏜 리슬링",70000.0,NaN,1.0,3.0,3.0,1.0,Wine Whiskey\n호주 Australia 화이트 와인 White Wine 그...
9,"보시오, 트로피칼 모스카토 패션 후르츠",24000.0,NaN,4.0,2.0,2.0,1.0,일상\n이마트24 편의점 달달한 스파클링 와인추천 [보시오 트로피칼 모스카토 패션후...


In [40]:
returnBlogReview(get_recommendations('달콤 상큼',df_winter_party).unique())

(202, 17954)


,name_ko,blogreview
0,크리스마스 아스티,크리스마스 트러스 생산 스파클링 레몬 라임 청포도 계열 과일 포 과 하지 않은 아스...
1,"에라스모, 로제 드 무르베드르",향 배 딸기 장미 트러스 오렌지 느낄 꽃 빛 과일 강하게 일어나며 그레이 프 푸르츠...
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",생산 호두 감초 타닌 오크 젤리 설탕 린 향 묵직한 무게 감 밸런스 5년 드 부드러...
3,"도미니오 데 라 베가, 디 엔드",발렌시아 포도 미와 신선한 산미 지역 달콤한 트러스 파인애플 입 가득 흰 풍부한 좋...
4,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
5,"떼땅져, 녹턴 시티 라이트 '루미너스'",건포도 복숭아 살구 꽃 트러스 향 과일 느껴지며 부드럽고 미감 샴페인 포도 스파클링...
6,"떼땅져, 녹턴 시티 라이트",건포도 복숭아 살구 꽃 트러스 향 과일 느껴지며 부드럽고 미감 샴페인 포도 스파클링...
7,"얄리, 프리미엄 릴리즈 언오크드 샤르도네",향 포도 허브 레몬 자몽 트러스 오렌지 미네랄 산뜻한 산도 접촉 생산 과일 바디 신...
8,"그랜트 버지, 쏜 리슬링",리 슬링 생산 포도 지역 미네랄 에덴 호주 바로 사 레몬 꽃 좋고 5 것 그랜트 버...
9,"보시오, 트로피칼 모스카토 패션 후르츠",모스 카토 발효 꽃 과실 도 청포도 향 긋한 미를 과정 과일 24시간 만든 인기 사...


In [41]:
# 겨울, 파티, 드라이
winter_party_dry = returnWineInfo(get_recommendations('드라이 무겁',df_winter_party).unique())
winter_party_dry

(202, 17954)


,name_ko,price,type,sugar,acidity,body,tannin,blog_text
0,"나파 하이랜즈, 샤도네이",124000.0,NaN,1.0,3.0,4.0,1.0,Tasting＆Food~♪\n서래마을 맛집<갓포아키x온> @ 와인모임\n학동와인 ・...
1,"카판넬레, 로사토",75000.0,NaN,1.0,4.0,3.0,1.0,BEST와인추천\n와인21닷컴이 선정한 3월 이달의 와인\n와인21 ・ 2021. ...
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",65000.0,NaN,1.0,3.0,4.0,3.0,프랑스와인\n샤또 롱보 꼬뜨 뒤 루씨옹 뀌베 삐에르 드 라 파브레귀2016(Chat...
3,"아그리콜라 푸니카, 바루아",140000.0,NaN,1.0,3.0,4.0,4.0,"Agricola Punica, Barrua, Isola dei Nuraghi, Sa..."
4,"메이오미, 피노 누아",90000.0,NaN,1.0,3.0,3.0,3.0,"Tasting Diary\nMeiomi, Pinot Noir 2018 (메이오미, ..."
5,"레이몬드, 리저브 셀렉션 메를로",190000.0,NaN,1.0,2.0,4.0,3.0,wine\n40주년 한정판 와인 : 레이몬드 리저브 셀렉션 까베르네 소비뇽 2018...
6,"제라르 베르트랑, 꼬뜨 로즈 로제",33000.0,NaN,1.0,3.0,3.0,1.0,ROSE WINE\n[Gerard Bertrand] Cote des Roses Ro...


In [42]:
returnBlogReview(get_recommendations('드라이 무겁',df_winter_party).unique())

(202, 17954)


,name_ko,blogreview
0,"나파 하이랜즈, 샤도네이",나파 샤 아몬드 레몬 오크 과실 뛰어난 포도 원 청사 라임 반짝이는 익은 미와 미네...
1,"카판넬레, 로사토",배 복숭아 딸기 꽃 향 토스카나 허브 민트 보여지는 은은하게 신선한 과실 장미꽃 연...
2,"샤또 롱보, 뀌베 삐에르 드 라 파브레귀",생산 호두 감초 타닌 오크 젤리 설탕 린 향 묵직한 무게 감 밸런스 5년 드 부드러...
3,"아그리콜라 푸니카, 바루아",자두 체리 블랙베리 감초 허브 세이지 로즈마리 향 완벽한 익은 야생 블랙 올리브 미...
4,"메이오미, 피노 누아",카운티 누 빈야드 향 자두 오크 캘리포니아 포도 양조 진행 는 와 체리 딸기 홍차 ...
5,"레이몬드, 리저브 셀렉션 메를로",자두 블루베리 블랙 커런트 오크 18 개월 정도 프랑스 산 통 숙성 리저 브 메를 ...
6,"제라르 베르트랑, 꼬뜨 로즈 로제",장미 은은한 향 체리 꽃 과실 싱그러운 하여 병 살구 살구색 콤 풍부한 붉은 향기 ...
